## Data Preparation for Regression Analysis

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# average number of OSM mappers per months since 2022
start_year = 2022
data_path = '/data/processing/ohsome-stats-by-year-full/*/*.parquet'

sql = f"""
COPY
(
    SELECT country_iso_a3, year, month, count(distinct user_id) as n_users
    FROM (
    	SELECT user_id, unnest(country_iso_a3) as country_iso_a3, year, month
    	FROM read_parquet("{data_path}", hive_partitioning=1)
    	WHERE year >= {start_year}
    ) foo
    GROUP BY year, month, country_iso_a3
    ORDER BY country_iso_a3, year, month
    )
TO 'mappers_per_country_per_month.csv' (HEADER, DELIMITER ',');
"""
print(sql)


COPY
(
    SELECT country_iso_a3, year, month, count(distinct user_id) as n_users
    FROM (
    	SELECT user_id, unnest(country_iso_a3) as country_iso_a3, year, month
    	FROM read_parquet("/data/processing/ohsome-stats-by-year-full/*/*.parquet", hive_partitioning=1)
    	WHERE year >= 2022
    ) foo
    GROUP BY year, month, country_iso_a3
    ORDER BY country_iso_a3, year, month
    )
TO 'mappers_per_country_per_month.csv' (HEADER, DELIMITER ',');



In [3]:
osm_df = pd.read_csv("../data/mappers_per_country_per_month.csv")
osm_df_avg = osm_df.groupby(["country_iso_a3"]).agg(
    avg_OSM_users_month=pd.NamedAgg(column="n_users", aggfunc="mean"),
    total_OSM_users=pd.NamedAgg(column="n_users", aggfunc="sum")
)
display(osm_df_avg.sort_values(by="avg_OSM_users_month", ascending=False)[0:15])

,avg_OSM_users_month,total_OSM_users
country_iso_a3,,
DEU,7300.05,146001
USA,4973.95,99479
FRA,3782.00,75640
GBR,2012.95,40259
ITA,2001.20,40024
RUS,1785.55,35711
POL,1591.75,31835
ESP,1306.75,26135
BRA,1172.00,23440


In [4]:
# load HDI data and get ISO A3 code
hdi_df = pd.read_json("../data/hdi_2021.json")
hdi_df["iso_a3_code"] = hdi_df["country"].str[:3]
hdi_df.set_index("iso_a3_code", inplace=True)
hdi_df.rename(columns={"value": "hdi_2021"}, inplace=True)
display(hdi_df)

,country,index,indicator,year,hdi_2021
iso_a3_code,,,,,
AFG,AFG - Afghanistan,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.478
AGO,AGO - Angola,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.586
ALB,ALB - Albania,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.796
AND,AND - Andorra,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.858
ARE,ARE - United Arab Emirates,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.911
...,...,...,...,...,...
ZZG,ZZG.ECA - Europe And Central Asia,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.796
ZZH,ZZH.LAC - Latin America And The Caribbean,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.754
ZZI,ZZI.SA - South Asia,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.632


In [5]:
pop_df = pd.read_csv("../data/population.csv", delimiter=';')
pop_df.set_index("Country Code", inplace=True)
pop_df.rename(columns={"2022": "pop_2022"}, inplace=True)
pop_df.drop(columns=["Unnamed: 3", "Unnamed: 4", "Country Name"], inplace=True)
display(pop_df)

,pop_2022
Country Code,
ABW,106445.0
AFE,720859132.0
AFG,41128771.0
AFW,490330870.0
AGO,35588987.0
...,...
XKX,1761985.0
YEM,33696614.0
ZAF,59893885.0


In [6]:
disaster_df = pd.read_csv("../data/deaths_per_100000.csv", delimiter=';', decimal=",")
disaster_df.set_index("ID_2", inplace=True)
disaster_df.rename(columns={"Deaths per 100000": "disaster_death_per_100k"}, inplace=True)
disaster_df.rename(columns={"Total Deaths": "disaster_death"}, inplace=True)
display(disaster_df)

,Country Name,disaster_death,World Bank population 2022,disaster_death_per_100k
ID_2,,,,
AFG,Afghanistan,5106,41128771,12.414667
AGO,Angola,220,35588987,0.618169
ALB,Albania,403,2777689,14.508464
ARG,Argentina,49,46234830,0.105981
ARM,Armenia,0,2780469,0.000000
...,...,...,...,...
WSM,Samoa,83,222382,37.323165
YEM,"Yemen, Rep.",1499,33696614,4.448518
ZAF,South Africa,877,59893885,1.464256


In [7]:
# OSM AI-generated data stats
ai_df = gpd.read_file("../data/blds_world-states-2024.geojson")
columns_to_keep = [
    "ISO_A3",
    "ai_blds_2024",
    "blds_total",
    "ai_percentage",
    "ai_per_roun"
]
ai_df.drop(
    ai_df.columns.difference(columns_to_keep),
    axis=1, 
    inplace=True
)
ai_df.set_index("ISO_A3", inplace=True)

ai_df["non_ai_blds_2024"] = ai_df["blds_total"] - ai_df["ai_blds_2024"]
display(ai_df)

,ai_blds_2024,blds_total,ai_percentage,ai_per_roun,non_ai_blds_2024
ISO_A3,,,,,
IDN,313388.0,39804168.0,0.787325,0.787,39490780.0
MYS,18849.0,1049053.0,1.796763,1.797,1030204.0
CHL,1408.0,674164.0,0.208851,0.209,672756.0
BOL,72868.0,361047.0,20.182414,20.182,288179.0
PER,10860.0,2052082.0,0.529219,0.529,2041222.0
...,...,...,...,...,...
BHR,100.0,18061.0,0.553679,0.554,17961.0
-99,0.0,47.0,0.000000,0.000,47.0
-99,0.0,NaN,NaN,NaN,NaN


In [16]:
# merge all dataframes
df = ai_df.merge(
    osm_df_avg[["total_OSM_users", "avg_OSM_users_month"]],
    how="left",
    left_index=True,
    right_index=True
).merge(
    pop_df["pop_2022"],
    how="left",
    left_index=True,
    right_index=True
).merge(
    hdi_df["hdi_2021"],
    how="left",
    left_index=True,
    right_index=True
).merge(
    disaster_df[["disaster_death_per_100k", "disaster_death"]],
    how="left",
    left_index=True,
    right_index=True
)

df.reset_index(inplace=True)

# fill Nan values
df_cleaned = df[
    (df["ISO_A3"] != '-99')
    &
    (df["pop_2022"].notnull())
]
df_cleaned.fillna(0, inplace=True)

# normalize values
columns = [
    "hdi_2021",
    "disaster_death_per_100k",
    "avg_OSM_users_month"
]

for column in columns:
    df_cleaned[f"{column}_norm"] = (df_cleaned[column]-df_cleaned[column].mean())/df_cleaned[column].std()

# save to csv
df_cleaned.to_csv("../data/regression_analysis_data.csv")
display(df_cleaned)

/tmp/ipykernel_78682/516587121.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.fillna(0, inplace=True)
/tmp/ipykernel_78682/516587121.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[f"{column}_norm"] = (df_cleaned[column]-df_cleaned[column].mean())/df_cleaned[column].std()
/tmp/ipykernel_78682/516587121.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

,ISO_A3,ai_blds_2024,blds_total,ai_percentage,ai_per_roun,non_ai_blds_2024,total_OSM_users,avg_OSM_users_month,pop_2022,hdi_2021,disaster_death_per_100k,disaster_death,hdi_2021_norm,disaster_death_per_100k_norm,avg_OSM_users_month_norm
0,IDN,313388.0,39804168.0,0.787325,0.787,39490780.0,15720.0,786.000000,275501339.0,0.705,2.540822,7000.0,0.079509,-0.174118,0.875872
1,MYS,18849.0,1049053.0,1.796763,1.797,1030204.0,4785.0,239.250000,33938221.0,0.803,0.409568,139.0,0.629993,-0.298756,-0.011980
2,CHL,1408.0,674164.0,0.208851,0.209,672756.0,3961.0,198.050000,19603733.0,0.855,2.423008,475.0,0.922086,-0.181008,-0.078883
3,BOL,72868.0,361047.0,20.182414,20.182,288179.0,2011.0,100.550000,12224110.0,0.692,2.347819,287.0,0.006486,-0.185405,-0.237211
4,PER,10860.0,2052082.0,0.529219,0.529,2041222.0,3581.0,179.050000,34049588.0,0.762,1.365655,465.0,0.399688,-0.242843,-0.109737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,NRU,0.0,1903.0,0.000000,0.000,1903.0,33.0,2.200000,12668.0,0.000,0.000000,0.0,-3.880600,-0.322708,-0.396918
201,FSM,0.0,8036.0,0.000000,0.000,8036.0,60.0,3.333333,114164.0,0.628,0.000000,0.0,-0.353013,-0.322708,-0.395078
202,VUT,0.0,98039.0,0.000000,0.000,98039.0,124.0,6.200000,326740.0,0.607,0.918161,3.0,-0.470974,-0.269013,-0.390423
203,PLW,0.0,2058.0,0.000000,0.000,2058.0,54.0,2.842105,18055.0,0.767,0.000000,0.0,0.427774,-0.322708,-0.395876


# Regression Analysis
The analysis is performed with R.

## AI-generated Buildings

In [10]:
from rpy2.robjects.packages import importr
import rpy2.robjects as ro

In [17]:
out = ro.r(f'''
    library(magrittr) # needs to be run every time you start R and want to use %>%
    library(dplyr)    # alternatively, this also loads %>%
    library(tidyr)
    
    dat <- read.csv("../data/regression_analysis_data.csv")

    g_AI <- glm(
     ai_blds_2024 ~
        hdi_2021_norm +
        disaster_death_per_100k_norm +
        avg_OSM_users_month_norm +
        log(pop_2022),
    family = quasipoisson,
    data = dat
    )
    summary(g_AI)
''')

print(out)

out = ro.r(f'''
1 - g_AI$deviance / g_AI$null.deviance
''')
print(out)
print("""
######################################################################
""")


Call:
glm(formula = ai_blds_2024 ~ hdi_2021_norm + disaster_death_per_100k_norm + 
    avg_OSM_users_month_norm + log(pop_2022), family = quasipoisson, 
    data = dat)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-773.08  -208.87   -96.97   -23.56  1934.72  

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                   -5.0608     2.2168  -2.283   0.0236 *  
hdi_2021_norm                  0.2094     0.2303   0.909   0.3645    
disaster_death_per_100k_norm   0.2378     0.1040   2.286   0.0235 *  
avg_OSM_users_month_norm      -0.2224     0.2391  -0.930   0.3537    
log(pop_2022)                  0.9288     0.1243   7.472 3.56e-12 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for quasipoisson family taken to be 218344.4)

    Null deviance: 37443638  on 180  degrees of freedom
Residual deviance: 19076233  on 176  degrees of freedom
AIC: NA

Number of Fisher Scorin

## Non-AI Buildings

In [18]:
out = ro.r(f'''
    library(magrittr) # needs to be run every time you start R and want to use %>%
    library(dplyr)    # alternatively, this also loads %>%
    library(tidyr)
    
    dat <- read.csv("../data/regression_analysis_data.csv")

    g_non_AI <- glm(
     non_ai_blds_2024 ~
        hdi_2021_norm +
        disaster_death_per_100k_norm +
        avg_OSM_users_month_norm +
        log(pop_2022),
    family = quasipoisson,
    data = dat
    )
    summary(g_non_AI)
''')

print(out)

out = ro.r(f'''
1 - g_non_AI$deviance / g_non_AI$null.deviance
''')
print(out)
print("""
######################################################################
""")


Call:
glm(formula = non_ai_blds_2024 ~ hdi_2021_norm + disaster_death_per_100k_norm + 
    avg_OSM_users_month_norm + log(pop_2022), family = quasipoisson, 
    data = dat)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3970.9   -858.7   -257.9    214.3   6298.2  

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                   4.20955    0.78980   5.330 2.98e-07 ***
hdi_2021_norm                 0.22965    0.09138   2.513   0.0129 *  
disaster_death_per_100k_norm  0.01448    0.08096   0.179   0.8582    
avg_OSM_users_month_norm      0.16585    0.02315   7.165 2.04e-11 ***
log(pop_2022)                 0.61333    0.04451  13.780  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for quasipoisson family taken to be 1816477)

    Null deviance: 970940917  on 180  degrees of freedom
Residual deviance: 291382315  on 176  degrees of freedom
AIC: NA

Number of Fisher S